In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from datetime import datetime
from sklearn.metrics import jaccard_score,accuracy_score,f1_score
import tensorflow as tf
import os
from PIL import Image as im
from google.colab import drive


In [ ]:


def read_input(file_):
    img = cv2.imread(folder + 'train_partition_images/full_color_training_' + str(file_) + '.png') #full colour rgb image ( 3 dimensional )
    #print(img)
    img = np.reshape(img, (256*256,3)) 
    #print("img_dim: ",img.ndim) 
    return img

def read_test_input(file_):
    img = cv2.imread(folder + 'test_partition_images/full_color_training_' + str(file_) + '.png') #full colour rgb image ( 3 dimensional )
    #print(img)
    img = np.reshape(img, (256*256,3)) 
    #print("img_dim: ",img.ndim) 
    return img

In [ ]:
def read_labels(file_):
    label_img = cv2.imread(folder + 'train_partition_ground_truth/ground_truth_training_' + str(file_) + '.png')
    #print("label_img_before: ",label_img.ndim)
    label = label_img[:,:,0] #one channel
    labels = np.reshape(label, (256*256))
    #print(label)
    #print("label_dim: ",label.ndim)
    return labels

def read_test_labels(file_):
    label_img = cv2.imread(folder + 'test_partition_ground_truth/ground_truth_training_' + str(file_) + '.png')
    #print("label_img_before: ",label_img.ndim)
    label = label_img[:,:,0] #one channel
    labels = np.reshape(label, (256*256))
    #print(label)
    #print("label_dim: ",label.ndim)
    return labels

In [ ]:
def to_string(files,i):
    return files['name'][i]


In [ ]:
start_time = datetime.now()
print(str(start_time))


2021-07-10 23:24:01.888219


In [ ]:
folder='/content/drive/MyDrive/SequoiaMulti_30/'


In [ ]:
files =pd.read_csv(folder + 'train.csv')


In [ ]:
#image=cv2.imread('/content/drive/MyDrive/SequoiaMulti_30/train_partition_images/full_color_training_1.png')
#print(image)
#print(image.shape)
#print(image[:256,:256,0])
#cv2_imshow(image[:256,:256,])

In [ ]:
num = to_string(files,0)
X_train = read_input(num)
y_train = read_labels(num)

In [ ]:
model = SGDClassifier(loss='hinge',fit_intercept=False)

In [ ]:
for file_ in range(1,len(files['name'])):
    num = to_string(files,file_)
    if(file_ == 1001 or file_ == 2001):
        X_train = read_input(num)
        y_train = read_labels(num)
        continue
        
    if(file_%100 == 0):
        print(num)
        #print(file_)
    #print(num)
    X_train = np.concatenate((X_train,read_input(num)))
    y_train = np.concatenate((y_train,read_labels(num)))
    #break
    if( file_%1000==0 and file_!=0):
        print('Done Loading:')
        model_log = model.partial_fit(X_train,y_train,classes=np.unique(y_train))
        #break




101
201
301
401
501
601
701
801
901
1001
Done Loading:
1101
1201
1301
1401
1501
1601
1701
1801


In [ ]:

print(X_train.shape)
print(y_train.shape)

pickle.dump(model_log, open(folder + 'linear_svm_' + str(file_) + '_images_fold_' + '.sav', 'wb'))




(57278464, 3)
(57278464,)


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
model = pickle.load(open(folder + 'linear_svm_1874_images_fold_.sav', 'rb'))


In [ ]:
test_files = pd.read_csv(folder + 'test.csv') 

In [ ]:
num = to_string(test_files,0)
X_test = read_test_input(num)
y_test = read_test_labels(num)

In [ ]:
predictions_folder = folder + 'Predictions_' + '/'
if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)

In [ ]:
height=256
width=256
final_img = 0
points = height*width

In [ ]:
for file_ in range(0,len(test_files['name'])):
    num = to_string(test_files,file_)
    
        
    #if(file_%100 == 0):
        #print(file_)
  
    y_pred = model.predict(read_test_input(num))
    y_pred_img = np.reshape(y_pred,(height,width,1))
    #print(y_pred_img.shape)
    
    final_img = np.stack((y_pred_img,y_pred_img,y_pred_img),axis=2)
    #print(final_img.shape)
    final_img = final_img[:,:,:,0]
    #print(final_img.shape)
    #print(np.unique(final_img))
    y_color = np.where(final_img==1,60,final_img)
    y_color = np.where(y_color==2,255,y_color)
    cv2.imwrite(predictions_folder + 'pred_' + str(num) + '.png', y_color)
    #print(np.unique(y_color))
    #plt.imshow(y_color)
    #plt.show()
    #if(file_ == 2):
    #  break
        


In [ ]:
end_time = datetime.now()
print('\nDuration: {}'.format(end_time - start_time))
print (str(end_time))